In [1]:
import os

import numpy as np
import scipy as sp
import matplotlib.pyplot as plt

from osgeo import gdal
from scipy.io import loadmat
from scipy.fft import ifft, fft, fftshift, ifftshift, fftfreq
from scipy.fft import ifft2, fft2
from scipy.constants import pi, c

In [145]:
PRIMARY_HEADER = [
    3,   # Packet Version Number
    1,   # Packet Type
    1,   # Secondary Header Flag
    7,   # Process ID
    4,   # Process Category
    2,   # Sequence Flags
    14,  # Packet Sequence Count 
    16   # Packet Data Length
]

PRIMARY_HEADER_FIELDS = [
    'packet_version_number',
    'packet_type',
    'secondary_header_flag',
    'process_id',
    'process_category',
    'sequence_flags',
    'packet_sequence_count',
    'packet_data_length'
]

SECONDARY_HEADER = [
    32,  # Coarse Time
    16,  # Fine Time
    32,  # Sync Marker
    32,  # Data Take ID
    8,   # ECC Number
    1,   # N/A
    3,   # Test Mode
    4,   # RX Channel ID
    32,  # Instrument Configuration ID
    8,   # Sub-Commutative Ancillary Data Word Index
    16,  # Sub-Commutative Ancillary Data Word
    32,  # Counter Service
    32,  # PRI Count
    1,   # Error Flag
    2,   # N/A
    5,   # BAQ Mode
    8,   # BAQ Block Length
    8,   # N/A
    8,   # Range Decimation
    8,   # RX Gain
    16,  # TX Ramp Rate
    16,  # Pulse Start Frequency
    24,  # Pulse Length
    3,   # N/A
    5,   # Rank
    24,  # PRI
    24,  # SWST
    24,  # SWL
    24,  # SAS SSB Message *** TODO: See Page 43, 44
    24,  # SES SSB Message *** TODO: See Page 50
    16,  # Number of Quadratures
    8,   # N/A
]

SECONDARY_HEADER_FIELDS = [
    'coarse_time',
    'fine_time',
    'sync_marker',
    'data_take_id',
    'ecc_number',
    'na_1',
    'test_mode',
    'rx_channel_id',
    'instrument_configuration_id',
    'sc_data_word_index',
    'sc_data_word',
    'counter_service',
    'pri_count',
    'error_flag',
    'na_2',
    'baq_mode',
    'baq_block_length',
    'na_3',
    'range_decimation',
    'rx_gain',
    'tx_ramp_rate',
    'pulse_start_frequency',
    'pulse_length',
    'na_4',
    'rank',
    'pri',
    'swst',
    'swl',
    'sas_ssb_message',
    'ses_ssb_message',
    'num_quadratures',
    'na_5',
]

SUB_COMMUTATIVE_DATA_SERVICE = [
    16,      # Dummy Data
    64,      # X-Axis ECEF Position
    64,      # Y-Axis ECEF Position
    64,      # Z-Axis ECEF Position
    32,      # X-velocity ECEF
    32,      # Y-velocity ECEF
    32,      # Z-velocity ECEF
    16,      # POD Data Stamp 1
    16,      # Pod Data Stamp 2
    16,      # Pod Data Stamp 3
    16,      # Pod Data Stamp 4
    32,      # Q0 Attitude Quaternion
    32,      # Q1 Attitude Quaternion
    32,      # Q2 Attitude Quaternion
    32,      # Q3 Attitude Quaternion
    32,      # OmegaX Angular Rate
    32,      # OmegaY Angular Rate
    32,      # OmegaZ Angular Rate
    16,      # Data Time Stamp 1
    16,      # Data Time Stamp 2
    16,      # Data Time Stamp 3
    16,      # Data Time Stamp 4
    16,      # Pointing Status
    16,      # Temperature Update Status
    8, 8, 8, # Tile 1 EFE H, V Temperature and Activate TA Temperature
    8, 8, 8, # Tile 2
    8, 8, 8, # Tile 3
    8, 8, 8, # Tile 4
    8, 8, 8, # Tile 5
    8, 8, 8, # Tile 6
    8, 8, 8, # Tile 7
    8, 8, 8, # Tile 8
    8, 8, 8, # Tile 9
    8, 8, 8, # Tile 10
    8, 8, 8, # Tile 11
    8, 8, 8, # Tile 12
    8, 8, 8, # Tile 13
    8, 8, 8, # Tile 14
    9,       # N/A
    7,       # TGU Temperature
]

In [156]:
class Packet:
    def __init__(
        self,
        primary_header,
        secondary_header,
        user_data_field
    ):

        
        self.primary_header   = primary_header
        self.secondary_header = secondary_header
        self.raw_user_data    = user_data_field
        self.user_data_field  = _decode_user_data_field()

    def _decode_user_data_field(self):
        return self.raw_user_data

In [161]:
def read_and_pop(bit_string: str, bit_length: int):
    return bit_string[0:bit_length], bit_string[bit_length:]


def get_header_dict(header_bytes, header_bit_lengths, header_field_names):
    bit_string = ''
    for byte in header_bytes:
        bit_string += '{:08b}'.format(byte)

    header_value_array = []
    for bit_len in header_bit_lengths:
        header_field_value, bit_string = read_and_pop(bit_string, bit_len)
        header_value_array.append(header_field_value) 

    index = 0
    header_dict = {}
    for field in header_field_names:
        if index >= len(header_field_names) or index >= len(header_value_array):
            raise ValueError('The number of field names and header values does not match.')
        header_dict[field] = header_value_array[index]
        index += 1

    return header_dict


def read_packets(l0_data_file):
    packets = []
    with open(l0_data_file, 'rb') as raw_data:
        primary_header_bytes = raw_data.read(6)
        primary_header = get_header_dict(primary_header_bytes, PRIMARY_HEADER, PRIMARY_HEADER_FIELDS)
        secondary_header_bytes = raw_data.read(62)
        secondary_header = get_header_dict(secondary_header_bytes, SECONDARY_HEADER, SECONDARY_HEADER_FIELDS)
        user_data_section = None
        return primary_header, secondary_header

In [162]:
l0_data_file = 'sar_data/S1A_IW_RAW__0SDV_20240806T135224_20240806T135256_055093_06B68A_AE41.SAFE/s1a-iw-raw-s-vv-20240806t135224-20240806t135256-055093-06b68a.dat'

In [163]:
primary_header, secondary_header = read_packets(l0_data_file)

In [164]:
primary_header

{'packet_version_number': '000',
 'packet_type': '0',
 'secondary_header_flag': '1',
 'process_id': '1000001',
 'process_category': '1100',
 'sequence_flags': '11',
 'packet_sequence_count': '10101110010101',
 'packet_data_length': '0100100010111101'}

In [165]:
secondary_header

{'coarse_time': '01010011110111001110110100101010',
 'fine_time': '0001111001001101',
 'sync_marker': '00110101001011101111100001010011',
 'data_take_id': '00001101011011010001010111000000',
 'ecc_number': '00001000',
 'na_1': '0',
 'test_mode': '000',
 'rx_channel_id': '0000',
 'instrument_configuration_id': '00000000000000000000000000000111',
 'sc_data_word_index': '00011011',
 'sc_data_word': '1011111011000011',
 'counter_service': '00000000000000111010101110010101',
 'pri_count': '00000000000000111011011010111000',
 'error_flag': '0',
 'na_2': '00',
 'baq_mode': '01100',
 'baq_block_length': '00011111',
 'na_3': '00000000',
 'range_decimation': '00001000',
 'rx_gain': '00001000',
 'tx_ramp_rate': '1000011001000101',
 'pulse_start_frequency': '0011000000101111',
 'pulse_length': '000000000000011110101111',
 'na_4': '000',
 'rank': '01001',
 'pri': '000000000101010101100011',
 'swst': '000000000000111001100001',
 'swl': '000000000011011010011011',
 'sas_ssb_message': '011111000110000